# IDL Assignment 3
Members:

Harsh Solanki (226254)

Vaibhav Bhatia (226222)

# Part 1 : MNIST

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.models import Model
tf.__version__



'2.2.0'

In [0]:

mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


data = tf.data.Dataset.from_tensor_slices(
    (train_images.reshape([-1, 28, 28, 1]).astype(np.float32) / 255, train_labels.astype(np.int32)))
data = data.shuffle(buffer_size=60000).batch(128).repeat()

test_data = tf.data.Dataset.from_tensor_slices(
    (test_images.reshape([-1, 28, 28, 1]).astype(np.float32) / 255, test_labels.astype(np.int32))).batch(10000)

train_steps = 1000

In [0]:
visible = tf.keras.layers.Input(shape=(28,28,1))
conv1 = Conv2D(32, kernel_size=7, activation='relu')(visible)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, kernel_size=4, activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

flat = Flatten()(pool2)
hidden1 = Dense(100, activation='relu')(flat)
hidden1 = Dropout(0.2)(hidden1)
output = Dense(10, activation='sigmoid')(hidden1)
model = Model(inputs=visible, outputs=output)
# summarize layers
print(model.summary())

# Adam makes things much smoother
opt = tf.optimizers.Adam()
# from_logits = True!! #neverforget
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])
varis = model.trainable_variables


Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 22, 22, 32)        1600      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 8, 8, 64)          32832     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)               1025

In [0]:
# this basically hasn't changed
for step, (img_batch, lbl_batch) in enumerate(data):
    if step > train_steps:
        break

    with tf.GradientTape() as tape:
        logits = model(img_batch)
        xent = loss_fn(lbl_batch, logits)

    grads = tape.gradient(xent, varis)
      
    opt.apply_gradients(zip(grads, varis))
    
    if not step % 100:
        preds = tf.argmax(logits, axis=1, output_type=tf.int32)
        acc = tf.reduce_mean(tf.cast(tf.equal(preds, lbl_batch),
                             tf.float32))
        print("Loss: {} Accuracy: {}".format(xent, acc))
    

Loss: 2.3036880493164062 Accuracy: 0.0859375
Loss: 1.5713406801223755 Accuracy: 0.890625
Loss: 1.5020711421966553 Accuracy: 0.9609375
Loss: 1.4846410751342773 Accuracy: 0.984375
Loss: 1.4801596403121948 Accuracy: 0.984375
Loss: 1.4793096780776978 Accuracy: 0.9765625
Loss: 1.500380516052246 Accuracy: 0.9609375
Loss: 1.4978487491607666 Accuracy: 0.9609375
Loss: 1.5029799938201904 Accuracy: 0.96875
Loss: 1.4759957790374756 Accuracy: 0.984375
Loss: 1.4984683990478516 Accuracy: 0.9609375


In [0]:
big_test_batch = next(iter(test_data))
test_preds = tf.argmax(model(big_test_batch[0]), axis=1,
                       output_type=tf.int32)
acc = tf.reduce_mean(tf.cast(tf.equal(test_preds, big_test_batch[1]),
                             tf.float32))
print(acc)

# model.evaluate(test_images, test_labels, batch_size=10000)

tf.Tensor(0.9812, shape=(), dtype=float32)


# Part 2 : CIFAR Implementation

Same archihtecture used in MNIST doesn't work.

Pending: Try different architectures and analyse the modifications with tensorboard




In [0]:
cifar = tf.keras.datasets.cifar10

(train_images, train_labels), (test_images, test_labels) = cifar.load_data()


data = tf.data.Dataset.from_tensor_slices(
    (train_images.reshape([-1, 32, 32, 3]).astype(np.float32) / 255, train_labels.astype(np.int32)))
data = data.shuffle(buffer_size=60000).batch(128).repeat()

test_data = tf.data.Dataset.from_tensor_slices(
    (test_images.reshape([-1, 32, 32, 3]).astype(np.float32) / 255, test_labels.astype(np.int32))).batch(10000)


train_labels = train_labels.reshape((-1))
test_labels = test_labels.reshape((-1))

In [0]:
visible = tf.keras.layers.Input(shape=(32,32,3))
conv1 = Conv2D(32, kernel_size=7, activation='relu')(visible)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, kernel_size=4, activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, kernel_size=(3,3), activation='relu')(pool2)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

flat = Flatten()(pool3)
hidden1 = Dense(100, activation='relu')(flat)
hidden1 = Dropout(0.2)(hidden1)
output = Dense(10, activation='sigmoid')(hidden1)
model = Model(inputs=visible, outputs=output)
# summarize layers
print(model.summary())

# Adam makes things much smoother
opt = tf.optimizers.Adam()
# from_logits = True!! #neverforget
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])
varis = model.trainable_variables

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 26, 26, 32)        4736      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 10, 10, 64)        32832     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 3, 3, 128)         73856     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 1, 1, 128)         0   

In [0]:
for step, (img_batch, lbl_batch) in enumerate(data):
    if step > train_steps:
        break

    with tf.GradientTape() as tape:
        logits = model(img_batch)
        xent = loss_fn(lbl_batch, logits)

    grads = tape.gradient(xent, varis)
      
    model.optimizer.apply_gradients(zip(grads, varis))
    
    if not step % 100:
        preds = tf.argmax(logits, axis=1, output_type=tf.int32)
        acc = tf.reduce_mean(tf.cast(tf.equal(preds, lbl_batch),
                             tf.float32))
        print("Loss: {} Accuracy: {}".format(xent, acc))

big_test_batch = next(iter(test_data))
test_preds = tf.argmax(model(big_test_batch[0]), axis=1,
                       output_type=tf.int32)
acc = tf.reduce_mean(tf.cast(tf.equal(test_preds, big_test_batch[1]),
                             tf.float32))
print(acc)

# model.evaluate(test_images, test_labels, batch_size=10000)

Loss: 2.3025529384613037 Accuracy: 0.107666015625
Loss: 2.0865719318389893 Accuracy: 0.106689453125
Loss: 1.9736580848693848 Accuracy: 0.099365234375
Loss: 2.0411219596862793 Accuracy: 0.08984375
Loss: 1.9038348197937012 Accuracy: 0.115234375
Loss: 1.9602935314178467 Accuracy: 0.1043701171875
Loss: 1.982086420059204 Accuracy: 0.1048583984375
Loss: 1.9040179252624512 Accuracy: 0.0906982421875
Loss: 1.9058732986450195 Accuracy: 0.10345458984375
Loss: 1.9699639081954956 Accuracy: 0.0953369140625
Loss: 1.8816304206848145 Accuracy: 0.098876953125
tf.Tensor(0.1, shape=(), dtype=float32)
